In [ ]:
import imageio as imgio
import numpy as np
from collections import deque
from PIL import Image, ImageDraw
import time
import heapq

In [ ]:
#Path Finding using Breadth First Search

def BFSfinder(img_path,start_point,end_point):                   #Breadth First Search
    img = imgio.imread(img_path, mode='L')
    bw_img = np.where(img < 128,0,255).astype(np.uint8)          #converts to black and white
    h, w = bw_img.shape
    x1, y1 = start_point
    x2, y2 = end_point
    
    if not(0<=x1<w and 0<=y1<h and bw_img[x1,y1]==0):            #checks if both points are on white pixels
        return False, None
    if not(0<=x2<w and 0<=y2<h and bw_img[x1,y1]==0):
        return False, None
    
    bricks = set([x1, y1])
    queue = deque([(x1, y1)])
    #movement = [(1,0), (0,1), (-1,0),(0,-1),(1,1), (-1,1), (-1,1),(-1,-1)] #allows diagonal pathing
    movement = [(1,0), (0,1), (-1,0),(0,-1)]                                #conventional 4 way pathing
    tracker = {}
    
    while queue:                                                 #BFS search using a double ended queue
        x, y = queue.popleft()
        if (x,y)==(x2,y2):
            path = [(x,y)]
            while (x,y)!=(x1,y1):
                x,y = tracker[(x,y)]
                path.append((x,y))
            path.reverse()
            return True, path
        
        for mx, my in movement:
            nx, ny = x+mx, y+my
            if(0<=nx<w and 0<ny<h and bw_img[ny,nx]==0 and (nx,ny) not in bricks):
                bricks.add((nx,ny))
                tracker[(nx,ny)]=(x,y)
                queue.append((nx,ny))
                
    return False, None

In [ ]:
#Path Finding using A*

def heur(a, b):                                              #Calculates Manhattan Distance
    return (abs(a[0]-b[0])+abs(a[1]-b[1]))
            
def nghbrs(x, y, w, h):                                      #Yields the neigbhours of the current pixel
    for mx, my in [(1,0), (0,1), (-1,0),(0,-1)]:
        nx, ny = x+mx, y+my
        if 0<=nx<w and 0<=ny<h:
            yield(nx,ny)

def Path(parent, current):                                   #Constructs the path from the current pixel
    path=[current]
    while current in parent:
        current=parent[current]
        path.append(current)
    path.reverse()
    return path
            
def black(pxl):                                              #checks if pixel is black
    return pxl == 0
            
def AStarfinder(img_path, start, end):
    
    img = imgio.imread(img_path, mode='L')
    bw_img = np.where(img < 128,0,255).astype(np.uint8)          #converts to black and white
    h, w = bw_img.shape
    
    if not (black(bw_img[start[1],start[0]])) or not (black(bw_img[end[1],end[0]])): #confirms if both points are black
        return False, None
    if not (0<=start[0]<w and 0<=start[1]<h or 0<=end[0]<w and 0<=end[1]<h):         #checks if points are within bounds
        return False, None

    queue = []                                                      #initalizing a heap to serve as a priority queue
    heapq.heappush(queue, (heur(start,end), 0, start))
    parent = {}                                                     #dictionary to store predecessors
    cur_cost = {start: 0}                                           #dictionary to store cheapest cost
    
    while queue:
        _, cost, current=heapq.heappop(queue)                       #pops node with lowest cost
        
        if current==end:
            return True, (Path(parent,current))                     #returns if goal is reached
        
        for nghbr in nghbrs(current[0], current[1], w, h):          #yields pixel neighbours and skips white pixels
            if not black(bw_img[nghbr[1],nghbr[0]]):
                continue
            
            n_cost = cost+1                                         #calculates cost to reach this neigbour
            if nghbr not in cur_cost or n_cost<cur_cost[nghbr]:     #if pixel is new, or cheaper path is found, update dict
                cur_cost[nghbr] = n_cost
                p = n_cost + heur(nghbr,end)
                heapq.heappush(queue, (p, n_cost, nghbr))
                parent[nghbr]=current
    return False, None

In [ ]:
#Draw tool for Visualization

def Img_output(img_path, path, color=(0,255,0)):
    
    img_color=imgio.imread(img_path)
    if len(img_color.shape)==2:                                 #Converts to RGB
        img_color = np.stack([img_color]*3, axis=-1)
    
    output = Image.fromarray(img_color)
    draw = ImageDraw.Draw(output)
    
    for x, y in path:
        draw.point((x,y), fill=color)
    return output

In [ ]:
#List of test points for the attached "PF_Test.jpg" file

#Accesible Black points:  (0-60,0-60),(560-600,560-600),(500,350),(350,500)
#Inaccesible Black point: (400,400)
#White points:            (500,150),(150,450)

In [ ]:
#Intilization block - Please edit accordingly

img_path = "D:/2D PathFinder/PF_Test.jpg"
start_point1 = (1,1)
end_point1 = (580,580)
#start_point2 = (100,400)               #Uncomment for double pathing problem
#end_point2 = (550,100)

In [ ]:
#Single Pathfinder with Visualization

tic = time.time()                                                           #Time calculation
#result, path = BFSfinder(img_path,start_point1,end_point1)                 #Uncomment for Breadth First
result, path = AStarfinder(img_path,start_point1,end_point1)               #Uncomment for A* search
toc = time.time()
if (result):
    print("Path exists.")
    print("Time taken:",(toc-tic))
    print ("Path taken:", path)
    output = Img_output(img_path, path)
    output.show()
else:
    print("Path does not exist.")
    print("Time taken:",(toc-tic))

In [ ]:
#Double Pathfinder with Visualization using BFS

tic = time.time()                                             #Time calculation
result1, path1 = BFSfinder(img_path,start_point1,end_point1)
result2, path2 = BFSfinder(img_path,start_point2,end_point2)
if (result1) and (result2):                                         #checks if paths exist for both sets
    mod = Img_output(img_path, path1, color=(255,255,255))          #modifies image to flag pixels on the first path
    mod.save("mod.jpg")
    final, path = BFSfinder("mod.jpg",start_point2,end_point2)      #finds second path on mod image
    toc = time.time()
else:
    print("Atleast 1 path does not exist")
    
if (final):
    print("Uninteresecting paths exist.")
    print("Time taken:",(toc-tic))
    output = Img_output(img_path, path1, color=(0,255,0))
    output.save("mod.jpg")
    output = Img_output("mod.jpg", path, color=(0,0,255))
    output.show()
else:
    print("Unintersecting paths do not exist")

    